In [1]:
import pandas as pd

In [2]:
data_dir = ['./data/trial3/', './data/trial4/', './data/trial5/']

In [3]:
df = pd.DataFrame()
for location in data_dir:
    df_ = pd.read_csv(location+'driving_log.csv', names=['center_cam', 'left_cam', 'right_cam', 'steering', 'throttle', 'brake', 'speed'])
    # Rename data location
    df_.iloc[:,0:3] = df_.iloc[:,0:3].apply(lambda x: location + 'IMG/' + x.str.split('\\').str[-1])
    # Append to the training dataframe
    df = pd.concat([df,df_])

In [4]:
df.head()

,center_cam,left_cam,right_cam,steering,throttle,brake,speed
0,./data/trial3/IMG/center_2021_01_06_19_41_25_4...,./data/trial3/IMG/left_2021_01_06_19_41_25_443...,./data/trial3/IMG/right_2021_01_06_19_41_25_44...,0.0,0.0,0.0,0.000082
1,./data/trial3/IMG/center_2021_01_06_19_41_25_5...,./data/trial3/IMG/left_2021_01_06_19_41_25_546...,./data/trial3/IMG/right_2021_01_06_19_41_25_54...,0.0,0.0,0.0,0.000078
2,./data/trial3/IMG/center_2021_01_06_19_41_25_6...,./data/trial3/IMG/left_2021_01_06_19_41_25_649...,./data/trial3/IMG/right_2021_01_06_19_41_25_64...,0.0,0.0,0.0,0.000082
3,./data/trial3/IMG/center_2021_01_06_19_41_25_7...,./data/trial3/IMG/left_2021_01_06_19_41_25_752...,./data/trial3/IMG/right_2021_01_06_19_41_25_75...,0.0,0.0,0.0,0.000078
4,./data/trial3/IMG/center_2021_01_06_19_41_25_8...,./data/trial3/IMG/left_2021_01_06_19_41_25_857...,./data/trial3/IMG/right_2021_01_06_19_41_25_85...,0.0,0.0,0.0,0.000079


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15195 entries, 0 to 7327
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   center_cam  15195 non-null  object 
 1   left_cam    15195 non-null  object 
 2   right_cam   15195 non-null  object 
 3   steering    15195 non-null  float64
 4   throttle    15195 non-null  float64
 5   brake       15195 non-null  float64
 6   speed       15195 non-null  float64
dtypes: float64(4), object(3)
memory usage: 949.7+ KB


In [6]:
df_clean = pd.DataFrame()
# Series with all camera angles
df_clean['camera'] = pd.concat([df['center_cam'], df['left_cam'], df['right_cam']])
# Series with all steering angels (and modified steering angles)
steering_offset = 0.02
df_clean['steering'] = pd.concat([
    df['steering'], 
    df['steering'].apply(lambda x: x + steering_offset),
    df['steering'].apply(lambda x: x - steering_offset)
    ])

df_clean.reset_index(drop=True, inplace=True);

In [7]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45585 entries, 0 to 45584
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   camera    45585 non-null  object 
 1   steering  45585 non-null  float64
dtypes: float64(1), object(1)
memory usage: 712.4+ KB


In [8]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Generator

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import cv2

In [10]:
# Constant preprocessing parameters
sample_img = mpimg.imread(df_clean['camera'][0])
scale_percent = 80 # percent of original size
width = int(sample_img.shape[1] * scale_percent / 100)
height = int(sample_img.shape[0] * scale_percent / 100)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

def preprocess(image):
    '''
    Preprocesses a given RGB image for input into the model.
    :params image: RGB image as a numpy array
    '''
    normalized = image.copy()
    normalized[:,:,0] = clahe.apply(normalized[:,:,0])
    normalized[:,:,1] = clahe.apply(normalized[:,:,1])
    normalized[:,:,2] = clahe.apply(normalized[:,:,2])
    grayscale = cv2.cvtColor(normalized, cv2.COLOR_RGB2GRAY)
    downscaled = cv2.resize(grayscale, (width, height), interpolation=cv2.INTER_AREA)
    return downscaled

In [11]:
from tensorflow.keras.utils import Sequence

In [12]:
class Generator(Sequence):

    def __init__(self, samples, batch_size):
        self.samples = samples
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(len(self.samples) / self.batch_size).astype(np.int32)

    def __getitem__(self, idx):
        images = []
        angles = []
        # Read in images (and preprocess) and steering angles from the current batch
        self.samples['camera'][idx * self.batch_size:(idx + 1) *
        self.batch_size].apply(lambda x: images.append(preprocess(mpimg.imread(x))));
        self.samples['steering'][idx * self.batch_size:(idx + 1) *
        self.batch_size].apply(lambda x: angles.append(x));

        X_train = np.expand_dims(np.array(images), axis=3)
        y_train = np.array(angles)

        return tuple(shuffle(X_train, y_train))

In [13]:
# Split training and validation samples
train_samples, validation_samples = train_test_split(df_clean, test_size=0.2)

# Set batch size
batch_size=1024

# compile and train the model using the generator function
train_generator = Generator(train_samples, batch_size)
validation_generator = Generator(validation_samples, batch_size)

# Training

In [16]:
import tensorflow as tf

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Flatten, Dense, Conv2D, MaxPool2D, Dropout, Cropping2D  
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
tf.keras.backend.clear_session()

In [19]:
model = Sequential()
# Input Layer
model.add(Lambda(lambda x: x/255.0 -0.5, input_shape=(height, width, 1)))
model.add(Cropping2D(cropping=((40,10),(0,0))))
# Convolution 1
model.add(Conv2D(24, (5,5), strides=(2,2), activation='relu'))
model.add(MaxPool2D(strides=(1,1)))
# Convolution 2
model.add(Conv2D(36, (5,5), strides=(2,2), activation='relu'))
model.add(MaxPool2D(strides=(1,1)))
# Convolution 3
model.add(Conv2D(48, (5,5), strides=(2,2), activation='relu'))
# Convolution 4
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(strides=(1,1)))
# Convolution 5
model.add(Conv2D(64, (3,3), activation='relu'))
# Flatten and Dropout
model.add(Flatten())
model.add(Dropout(0.25))
# Fully Connected 1
model.add(Dense(400, activation='relu'))
# Fully Connected 2
model.add(Dense(100, activation='relu'))
# Fully Connected 3 with Dropout
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
# Output
model.add(Dense(1));

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 128, 256, 1)       0         
_________________________________________________________________
cropping2d (Cropping2D)      (None, 78, 256, 1)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 37, 126, 24)       624       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 125, 24)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 61, 36)        21636     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 60, 36)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 28, 48)         4

In [21]:
stopper = EarlyStopping(monitor='val_loss', min_delta=0.0003, patience=5)

In [22]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
history = model.fit_generator(train_generator, 
    steps_per_epoch=np.ceil(len(train_samples)/batch_size),
    validation_data=validation_generator,
    validation_steps=np.ceil(len(validation_samples)/batch_size),
    epochs=30,
    callbacks=[stopper])

Epoch 1/30
36/36 [==============================] - 69s 2s/step - loss: 0.0802 - acc: 0.0853 - val_loss: 0.0574 - val_acc: 0.0857
Epoch 2/30
36/36 [==============================] - 56s 2s/step - loss: 0.0582 - acc: 0.0856 - val_loss: 0.0486 - val_acc: 0.0865
Epoch 3/30
36/36 [==============================] - 56s 2s/step - loss: 0.0503 - acc: 0.0863 - val_loss: 0.0436 - val_acc: 0.0869
Epoch 4/30
36/36 [==============================] - 56s 2s/step - loss: 0.0461 - acc: 0.0868 - val_loss: 0.0403 - val_acc: 0.0877
Epoch 5/30
36/36 [==============================] - 56s 2s/step - loss: 0.0415 - acc: 0.0874 - val_loss: 0.0360 - val_acc: 0.0887
Epoch 6/30
36/36 [==============================] - 55s 2s/step - loss: 0.0402 - acc: 0.0879 - val_loss: 0.0369 - val_acc: 0.0884
Epoch 7/30
36/36 [==============================] - 57s 2s/step - loss: 0.0370 - acc: 0.0881 - val_loss: 0.0313 - val_acc: 0.0893
Epoch 8/30
36/36 [==============================] - 56s 2s/step - loss: 0.0340 - acc: 0.08

In [23]:
model.save('model_generator.h5')

In [24]:
# from tensorflow.keras.models import load_model

In [25]:
# model = load_model('model.h5')

In [26]:
# y_train[400]

In [27]:
# model.predict(np.expand_dims(X_train_modified[400], axis=0))

In [28]:
# %run drive.py model.h5

# Fine-Tuning
Instead of re-training the network from scratch, this section can be modified to train on additional data.

In [29]:
# data_dir = ['./data/trial3/', './data/trial4/', './data/trial5/']

In [30]:
# df = pd.DataFrame()
# for location in data_dir:
#     df_ = pd.read_csv(location+'driving_log.csv', names=['center_cam', 'left_cam', 'right_cam', 'steering', 'throttle', 'brake', 'speed'])
#     # Rename data location
#     df_.iloc[:,0:3] = df_.iloc[:,0:3].apply(lambda x: location + 'IMG/' + x.str.split('\\').str[-1])
#     # Append to the training dataframe
#     df = pd.concat([df,df_])

In [31]:
# df_clean = df.copy()

In [32]:
# images = []
# measurements = []

# df_clean['center_cam'].apply(lambda x: images.append(mpimg.imread(x)));
# df_clean['steering'].apply(lambda x: measurements.append(x));
# steering_offset = 0.02
# # Left Camera Modifications
# df_clean['left_cam'].apply(lambda x: images.append(mpimg.imread(x)));
# df_clean['steering'].apply(lambda x: measurements.append(x + steering_offset));
# # Right Camera Modifications
# df_clean['right_cam'].apply(lambda x: images.append(mpimg.imread(x)));
# df_clean['steering'].apply(lambda x: measurements.append(x - steering_offset));

# X_train = np.array(images)
# y_train = np.array(measurements)

In [33]:
# modified_images =[]

# scale_percent = 80 # percent of original size
# width = int(X_train[0].shape[1] * scale_percent / 100)
# height = int(X_train[0].shape[0] * scale_percent / 100)
# dim = (width, height)

# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# for image in X_train:
#     normalized = image.copy()
#     normalized[:,:,0] = clahe.apply(normalized[:,:,0])
#     normalized[:,:,1] = clahe.apply(normalized[:,:,1])
#     normalized[:,:,2] = clahe.apply(normalized[:,:,2])
#     grayscale = cv2.cvtColor(normalized, cv2.COLOR_RGB2GRAY)
#     downscaled = cv2.resize(grayscale, dim, interpolation=cv2.INTER_AREA)
#     modified_images.append(downscaled)

# X_train_modified = np.array(modified_images)
# # Adding a dimension for Keras processing purposes
# X_train_modified = np.expand_dims(X_train_modified, 3)

In [34]:
# from tensorflow.keras.models import load_model
# tf.keras.backend.clear_session()
# model = load_model('model.h5')

# log_dir = "logs\\fit\\"
# tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# stopper = EarlyStopping(monitor='val_loss', min_delta=0.0003, patience=5)


In [35]:
# model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train_modified, y_train, epochs=30, batch_size=256, validation_split=0.2, shuffle=True, callbacks=[tensorboard_callback, stopper])
# model.save('model_fine_tune.h5')